#**Loading Data and preprocessing**

In [ ]:
import pandas as pd

data = pd.read_csv('Twitter_Data.csv')
data = data.dropna()

#**Tokenizing the whole input column**

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import itertools

# Concatenate every element in the column to form a story like text data
res = [i for i in data['clean_text']]
X = " ".join(res)

# Tokenize the whole text data extracted
tokenizer = Tokenizer()
tokenizer.fit_on_texts([X])

total_words = len(tokenizer.word_index) + 1
print("Total no. of unique words in the whole book :", total_words)

# Printing the first 10 items in the tokenizer.word_index dictionary
print(dict(itertools.islice(tokenizer.word_index.items(), 10)), '...')

#**Converting text input sequences into sequences of the token numbers**

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# [modi, and, you, will, not, india] ==> [1, 3, 5, 7, 6, 9]
input_seq = []
for messages in data['clean_text']:
  input_seq.append(tokenizer.texts_to_sequences([messages])[0])

# Padding extra zeroes to the start of sentence and converting the whole thing to a numpy array
max_sequence_len = max([len(seq) for seq in input_seq])
input_sequences = np.array(pad_sequences(input_seq, maxlen = max_sequence_len, padding = 'pre'))

# Data to Train on (Note: We are one hot encoding the output i.e., 'y')
X = input_sequences
y = np.array(to_categorical(data['category'], num_classes = 3))

#**Creating the model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

# Simple LSTM model
'''model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len))
model.add(LSTM(150))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])'''

# Bidirectional LSTM model
model = Sequential([
    Embedding(total_words, 100, input_length = max_sequence_len),
    Bidirectional(LSTM(150, return_sequences=True)),
    Bidirectional(LSTM(150)),
    Dense(3, activation="softmax"),
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 52, 100)           11367900  
                                                                 
 bidirectional (Bidirection  (None, 52, 300)           301200    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 300)               541200    
 onal)                                                           
                                                                 
 dense (Dense)               (None, 3)                 903       
                                                                 
Total params: 12211203 (46.58 MB)
Trainable params: 12211203 (46.58 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#**Training the model**

In [ ]:
model.fit(X, y, epochs = 40, verbose = 1, batch_size = 1024, validation_split = 0.33)

Epoch 1/40
107/107 [==============================] - 34s 245ms/step - loss: 0.6437 - accuracy: 0.7144 - val_loss: 0.2643 - val_accuracy: 0.9142
Epoch 2/40
107/107 [==============================] - 22s 210ms/step - loss: 0.1463 - accuracy: 0.9549 - val_loss: 0.1932 - val_accuracy: 0.9387
Epoch 3/40
107/107 [==============================] - 22s 206ms/step - loss: 0.0841 - accuracy: 0.9758 - val_loss: 0.1929 - val_accuracy: 0.9421
Epoch 4/40
107/107 [==============================] - 21s 193ms/step - loss: 0.0578 - accuracy: 0.9835 - val_loss: 0.2223 - val_accuracy: 0.9345
Epoch 5/40
107/107 [==============================] - 20s 190ms/step - loss: 0.0470 - accuracy: 0.9868 - val_loss: 0.2240 - val_accuracy: 0.9430
Epoch 6/40
107/107 [==============================] - 19s 182ms/step - loss: 0.0369 - accuracy: 0.9896 - val_loss: 0.2204 - val_accuracy: 0.9458
Epoch 7/40
107/107 [==============================] - 19s 183ms/step - loss: 0.0289 - accuracy: 0.9913 - val_loss: 0.2086 - val_ac

#**Saving the model**

In [ ]:
from tensorflow.keras.saving import load_model
#model.save('sentiment_analysis.keras')
new_model = load_model('sentiment_analysis.keras')

#**Testing the model**

In [ ]:
# print 'y' to check how we created the output_sentiment dictionary
output_sentiment = {0: 'neutral', 1: 'positive', 2: 'negative'}
seed_text = ["He is the worst", "He is partly good partly bad", "He is the best"]

for message in seed_text:
  # Convert to token
  token_list = tokenizer.texts_to_sequences([message])[0]
  # Path sequences
  token_list = pad_sequences([token_list], maxlen = max_sequence_len, padding = 'pre')
  # Model prediction
  predicted = np.argmax(model.predict(token_list), axis=-1)
  print(predicted, output_sentiment[predicted[0]])

1/1 [==============================] - 1s 1s/step
[2] negative
1/1 [==============================] - 0s 20ms/step
[0] neutral
1/1 [==============================] - 0s 20ms/step
[1] positive
